In [ ]:
import random
import time
from multiprocessing import Pool, cpu_count
def get_first_handle(braid):
    """
    Finds and returns the first handle of the given braid.

    return:
        (p, q) : Where p, q are the start and end points of the handle.
    """
    def is_handle(braid, p, q):
        """
        Checks if (p, q) is a valid handle in the given braid.

        return:
            True/False
        """
        if braid[p] + braid[q] == 0:
            v = braid[(p+1):q]
            j = abs(braid[p])
            for k in v:
                if not (abs(k) < (j - 1) or abs(k) > j):
                    return False
            else:
                return True
        else:
            return False
    p, q = 0, 0
    for q in range(len(braid)):
        for k in range(q, (p-1), -1):
            if is_handle(braid, k, q):
                return (k, q)
    else:
        return None

def reduce(braid, p, q):
    """
    Applies one step of the alphabetical homomorphism on the given handle.

    Returns a new reduced braid.
    """
    new_handle = []
    j = abs(braid[p])
    e = j/braid[p]
    for letter in braid[p:q+1]:
        exp = abs(letter)/letter
        idx = abs(letter)
        if (idx != j) and (idx != (j + 1)):
            new_handle.append(exp*idx)
        elif idx == (j + 1):
            new_handle.append(-1*e*idx)
            new_handle.append(exp*j)
            new_handle.append(e*idx)
    new_generators = braid[:p] + new_handle + \
                        braid[q + 1:]
    return new_generators
def reduce_fully(b, N):
    B = BraidGroup(N)
    braid=list(b.Tietze())
    pq=get_first_handle(braid)
    while(pq!=None): 
        p,q=pq
        r=reduce(braid, p,q)
        pq=get_first_handle(r)
        braid=r
    return B(braid)

def randomBraid(N, l):
    B = BraidGroup(N)
    generators=list(range(-N+1,N))
    generators.remove(0)
    b=B([1])
    while(len(b.Tietze())!=l):
        liste=random.choices(generators, k=l)
        b=B(liste)
    return b

def randomBraidAi(N,i, l):
    B = BraidGroup(N)
    generators=[-i-4,-i-3,-i-1, -i,i, i+1, i+3, i+4]
#     generators.remove(0)
    b=B([1])
    while(len(b.Tietze())!=l):
        liste=random.choices(generators, k=l)
        b=B(liste)
    return b
def weight_function(list_elements, g, N):
    w=0
    for element in list_elements:
        w+=len((reduce_fully(element*g.inverse(), N)).Tietze())**2
    return w
def mean_set(N, list_elements, l):
    B = BraidGroup(N)
    generators=list(range(-N+1,N))
    generators.remove(0)
    get_meant_set=False
    tamp_word=reduce_fully(randomBraid(N, l), N)
    wp=weight_function(list_elements, tamp_word, N )
    i=0
    while(get_meant_set==False):
        old_wp=wp
        for e in generators:
            y=reduce_fully(B([e])*tamp_word, N)
            p=weight_function(list_elements,y, N )
            if wp>=p:
                tamp_word=y
                wp=p
            i+=1
        if (wp==old_wp):
            get_meant_set=True
    return tamp_word

def attack(x):
    success=0
    Execution_Time=0
    N=90
    L=8
    k=100
    r=reduce_fully(randomBraid(N,L), N)
    h=reduce_fully(randomBraid(N,L), N)
    liste=[]
    for i in range(k):
        s=reduce_fully(randomBraid(N, L), N)
        s=reduce_fully(h*s*~r*~h, N)
        liste.append(s)
    Start_Time = time.time()
    if len(reduce_fully((mean_set(N, liste, L)*r*h), N).Tietze())==0:
        success+=1
    Execution_Time+= time.time() - Start_Time
#     print(Execution_Time)
    return success,Execution_Time
if __name__ ==  '__main__': 
    num_processors = cpu_count()
    print(num_processors)
    n=1
    p=Pool(processes = num_processors)
    output = p.map(attack,[i for i in range(n)])
    taux=1.0*sum([success for (success,Execution_Time) in output])/n
    tmoy=sum([Execution_Time for (success,Execution_Time) in output])/n
    print(f"Taux: {taux} temps moyen: {tmoy}")
